# Lab 12: Computational Complexity
## Data Structures & Algorithms
### 08/05/2025

## Today

* [Polynomial-time reductions](#reduce)
* [Satisfiability](#satisfy)
* [P, NP, NP-complete, EXP](#problems)
* [Practical Applications](#applications)
* [Exercises](#exercises)

## Learning Objectives

By the end of this lab, you will be able to:

1. **Explain polynomial-time reductions** and their role in comparing problem difficulty
2. **Formulate problems as SAT instances** using Conjunctive Normal Form (CNF)
3. **Classify problems** into P, NP, and NP-complete categories
4. **Implement a brute-force SAT solver** and understand its exponential complexity
5. **Verify solutions in polynomial time** (demonstrating the P vs NP distinction)

We have seen some (more or less efficient) algorithms for a number of problems, and have talked about ways to measure their efficiency. 

It turns out that for **many** problems we still don't know if polynomial-time algorithms exist!

## Polynomial-time reductions <a class="anchor" id="reduce"></a>

Although we might not be able (or haven't yet managed to) show that efficient algorithms exist for some problems, there are still ways to explore problems that are computationally hard, by comparing the relative difficulty of different problems. This is done through something called **reduction**: if we have two problems X and Y, we can say that X is at least as hard as Y if a known way to solve X would mean that we could also solve Y.

More formally: Problem X polynomial-time reduces to problem Y if instances of problem X can be solved using:
* Polynomial number of standard computational steps, plus
* Polynomial number of calls to a blackbox that solves problem Y in constant time.

The following mean the same thing:
* X is polynomial-time reducible to Y
* $X \leq_P Y$
* Y is at least as hard as X wrt polynomial time

Conclusions that can be made if $X \leq_P Y$
* and Y can be solved in polynomial time -> then X can be solved in polynomial time.
* and X cannot be solved in polynomial time -> then Y cannot be solved in polynomial time.
* and $Y \leq_P X$ -> X can be solved in polynomial time if and only if Y can be

Even if we don't know how to solve X or Y efficiently: if there is an efficient algorithm for one, there also is one for the other one (helps us when we design algorithms)!

**Techniques for reduction:**

**1. showing that two problems are equivalent**

Consider the independent set problem (does a graph contain an independent set of nodes - set with no edges between any of them - of size at least k): no known polynomial-time algorithm, but also no proof that none exists! And consider another hard problem, the vertex cover problem (does a graph contain a vertex cover - set of nodes so that all edges have one end in the set - of size at most k). It can be shown that they are **equivalent**, by showing that solving each of them automatically solves the other.

**2. showing that one problem is a special case for another one**

Similarly, it can be shown that the vertex cover problem is a **special case** of something more general called set cover problem (does there exist a collection of at most k subsets whose union is equal to all elements in the entire set)

**3. encoding one problem so that it can be related (reduced) to another problem (see below)**

<details>
<summary><b>Comprehension Check:</b> If we know that Problem A reduces to Problem B ($A \leq_P B$), and we discover a polynomial-time algorithm for A, what can we conclude about B?</summary>

**Answer:** Nothing! The reduction $A \leq_P B$ tells us that B is *at least as hard* as A. If we solve A efficiently, we learn nothing about B. However, if we solve B efficiently, then A can also be solved efficiently (by using B as a subroutine).

**Key insight:** Reductions preserve hardness in one direction only. Think of it as: "If I can solve the hard problem B, I can definitely solve the easier problem A."
</details>

## Satisfiability <a class="anchor" id="satisfy"></a>

One way to derive a **reduction** (point 3. in the list above) is by using the concept of satisfiability. The idea is to translate/encode the components of a problem X so that they represent the elements in another problem Y. The **satisfiability problem (SAT)** is fundamental to computational complexity theory.

### Conjunctive Normal Form (CNF)

A Boolean formula is in **Conjunctive Normal Form (CNF)** if it is a conjunction (AND) of clauses, where each clause is a disjunction (OR) of literals:

$$(\ell_{1,1} \lor \ell_{1,2} \lor \ldots) \land (\ell_{2,1} \lor \ell_{2,2} \lor \ldots) \land \ldots$$

Where each literal $\ell$ is either a variable $x_i$ or its negation $\neg x_i$.

**Example:** $(x_1 \lor \neg x_2) \land (\neg x_1 \lor \neg x_3) \land (x_2 \lor \neg x_3)$

### The SAT Problem

Given a Boolean formula in CNF, the **satisfiability problem (SAT)** asks: does there exist an assignment of truth values to variables that makes the formula true?

- **3-SAT**: SAT where each clause has exactly 3 literals (NP-complete)
- **2-SAT**: SAT where each clause has exactly 2 literals (in P - polynomial time solvable!)

<details>
<summary><b>Comprehension Check:</b> Why is 2-SAT in P but 3-SAT is NP-complete?</summary>

**Answer:** The key difference is that 2-SAT clauses $(a \lor b)$ can be rewritten as implications: $\neg a \Rightarrow b$ and $\neg b \Rightarrow a$. This creates an **implication graph** where we can use strongly connected components (SCC) to detect contradictions in $O(n + m)$ time.

With 3 literals per clause, this implication structure breaks down — we can't reduce $(a \lor b \lor c)$ to simple implications. The extra literal creates exponentially more possibilities to check.
</details>

### Example: Meeting Scheduling

Suppose we want to schedule a meeting:
- Elisa can meet Monday, Wednesday, or Thursday: $(Mon \lor Wed \lor Thu)$
- Juan cannot meet Wednesday: $(\neg Wed)$
- Ali cannot meet Friday: $(\neg Fri)$
- Kim cannot meet Tuesday or Thursday: $(\neg Tue) \land (\neg Thu)$

The CNF formula is: $(Mon \lor Wed \lor Thu) \land (\neg Wed) \land (\neg Fri) \land (\neg Tue) \land (\neg Thu)$

**Satisfying assignment:** $Mon = True$, all others $False$ → Monday works!

### Cook-Levin Theorem

A foundational result in complexity theory:

> **SAT is NP-complete** (Cook 1971, Levin 1973)

This means:
1. SAT is in NP (we can verify a solution in polynomial time)
2. Every problem in NP can be reduced to SAT in polynomial time

**Why is this remarkable?** The theorem says that ANY computational problem where solutions can be verified quickly can be encoded as a SAT instance. This includes:
- Program verification: "Does this program halt within k steps?"
- Constraint satisfaction: "Can these constraints be satisfied?"
- Optimisation: "Is there a solution with value ≥ k?"

**Proof sketch:** Any NP problem has a polynomial-time verifier. We can encode:
- The verifier's computation as a Boolean circuit
- The circuit's correctness constraints as CNF clauses
- A satisfying assignment corresponds to a valid certificate

**Consequence:** If anyone finds a polynomial-time algorithm for SAT, then P = NP.

### The 3-SAT to Independent Set Reduction

One of the most elegant reductions shows **3-SAT $\leq_P$ Independent Set**. Here's how it works:

**Construction:** Given a 3-SAT formula with $k$ clauses:

1. **Create clause gadgets:** For each clause $(a \lor b \lor c)$, create 3 nodes (one per literal) connected in a triangle.

2. **Add conflict edges:** Connect nodes for contradictory literals ($x_i$ and $\neg x_i$) across different triangles.

**Example:** For formula $(x_1 \lor \neg x_2 \lor x_3) \land (\neg x_1 \lor x_2 \lor x_3)$:

```
Clause 1 triangle:        Clause 2 triangle:
    x₁                        ¬x₁
   /  \                      /   \
 ¬x₂---x₃                  x₂----x₃

Conflict edges (dashed): x₁----¬x₁, ¬x₂----x₂
```

**Why it works:**
- Triangle edges force us to pick **at most one** node per clause
- To get an independent set of size $k$, we need **exactly one** from each clause
- Conflict edges prevent picking both $x_i$ and $\neg x_i$
- Therefore: independent set of size $k$ ↔ satisfying assignment

### Implementing a Brute-Force SAT Solver

Let's implement a simple SAT solver to understand the exponential complexity. We represent clauses as lists of integers:
- Positive integer `i` means variable $x_i$
- Negative integer `-i` means $\neg x_i$

For example, $(x_1 \lor \neg x_2) \land (\neg x_1 \lor x_3)$ becomes `[[1, -2], [-1, 3]]`

In [ ]:
from itertools import product


def evaluate_clause(clause: list[int], assignment: tuple[bool, ...]) -> bool:
    """
    Check if a single clause is satisfied by the assignment.
    
    Args:
        clause: List of literals (positive = variable, negative = NOT variable)
        assignment: Tuple of boolean values for variables (0-indexed)
    
    Returns:
        True if at least one literal in the clause is True
    """
    for literal in clause:
        var_idx = abs(literal) - 1  # Variables are 1-indexed
        var_value = assignment[var_idx]
        # Positive literal: need var to be True
        # Negative literal: need var to be False
        if (literal > 0 and var_value) or (literal < 0 and not var_value):
            return True
    return False


def brute_force_sat(clauses: list[list[int]], num_vars: int) -> dict[int, bool] | None:
    """
    Brute-force SAT solver - tries all 2^n possible assignments.
    
    Time complexity: O(2^n * m) where n = num_vars, m = total literals
    
    Args:
        clauses: List of clauses, each a list of integers
        num_vars: Number of variables
    
    Returns:
        Satisfying assignment as {var: bool}, or None if unsatisfiable
    """
    # Try all 2^n possible truth assignments
    for assignment in product([False, True], repeat=num_vars):
        # Check if ALL clauses are satisfied
        if all(evaluate_clause(clause, assignment) for clause in clauses):
            return {i + 1: val for i, val in enumerate(assignment)}
    return None


# Example: (x1 OR NOT x2) AND (NOT x1 OR x3) AND (x2 OR x3)
clauses = [[1, -2], [-1, 3], [2, 3]]
num_vars = 3

result = brute_force_sat(clauses, num_vars)
print(f"Clauses: {clauses}")
print(f"Satisfying assignment: {result}")

In [ ]:
# Demonstrate exponential growth: time to check all assignments
import time

def count_assignments_checked(num_vars: int) -> int:
    """Count how many assignments we'd need to check in worst case."""
    return 2 ** num_vars

# Show exponential growth
print("Exponential growth of brute-force SAT:")
print("-" * 40)
for n in [5, 10, 15, 20, 25, 30]:
    assignments = count_assignments_checked(n)
    print(f"n={n:2d} variables: {assignments:>15,} assignments to check")

print("\nAt 1 billion checks/second:")
print(f"n=30: {2**30 / 1e9:.1f} seconds")
print(f"n=50: {2**50 / 1e9 / 3600 / 24 / 365:.1f} years")
print(f"n=100: {2**100 / 1e9 / 3600 / 24 / 365:.2e} years")

## P, NP, NP-complete, EXP <a class="anchor" id="problems"></a>

The way in which we distinguish between these types of problems is based on the difference between **finding** a solution and **checking** a given solution. (For example: for the Independent Set or 3-SAT problems, we do not know a polynomial-time algorithm to find solutions but we can check a proposed solution in polynomial time.)

### P (Polynomial Time) — EASY TO SOLVE

Decision problems with a polynomial-time **algorithm**. These are problems we can solve efficiently.

**Examples:** Sorting, shortest path, 2-SAT, primality testing

### NP (Nondeterministic Polynomial Time) — EASY TO CHECK

Decision problems with a polynomial-time **certifier** (a checking algorithm that, given a 'certificate' or proof, can verify if a solution is correct). 

The name "NP" comes from **N**ondeterministic **P**olynomial time — imagine a machine that can magically guess the right certificate, then verify it in polynomial time.

**Key insight:** $P \subseteq NP$ — if we can solve a problem in polynomial time, we can certainly verify a solution in polynomial time (just solve it again).

**The P vs NP Question:** Is there a problem in NP that is NOT in P? This is one of the most important open problems in computer science and mathematics. Most experts believe **P ≠ NP** (that some NP problems cannot be solved efficiently), but no one has proven this. A proof either way would win a $1 million Clay Millennium Prize.

<div>
   <img src="images/screenshot_np.png" width="500px">
</div>

### NP-complete — THE HARDEST PROBLEMS IN NP

An NP-complete problem X has two properties:
1. X is in NP (solutions can be verified in polynomial time)
2. Every problem in NP can be reduced to X in polynomial time ($Y \leq_P X$ for all $Y$ in NP)

**Examples:** 3-SAT, Independent Set, Vertex Cover, Traveling Salesman (decision version)

**Why this matters:**
- If ANY NP-complete problem has a polynomial-time algorithm → P = NP
- If ANY NP-complete problem is proven to have no polynomial-time algorithm → P ≠ NP
- In practice: if your problem is NP-complete, don't expect to find an efficient exact algorithm — use approximations or heuristics instead

### EXP (Exponential Time) — VERY HARD

Decision problems that require exponential time $O(2^{n^k})$ to solve.

We know: $P \subseteq NP \subseteq EXP$, and that $P \neq EXP$ (there are problems in EXP that are NOT in P).

### Polynomial-Time Verification (Certifiers)

The key insight of NP is that while **finding** a solution may be hard, **verifying** a proposed solution is easy. Let's implement certifiers for SAT and Independent Set to see this in action.

In [ ]:
def verify_sat(clauses: list[list[int]], assignment: dict[int, bool]) -> bool:
    """
    Verify a SAT solution in polynomial time - O(m) where m = total literals.
    
    This is the "certifier" that proves SAT is in NP.
    Given a proposed assignment, we can quickly check if it satisfies all clauses.
    """
    for clause in clauses:
        clause_satisfied = False
        for literal in clause:
            var = abs(literal)
            if var not in assignment:
                return False  # Variable not assigned
            # Check if this literal is true under the assignment
            if (literal > 0 and assignment[var]) or (literal < 0 and not assignment[var]):
                clause_satisfied = True
                break
        if not clause_satisfied:
            return False
    return True


def verify_independent_set(graph: dict[int, set[int]], subset: set[int], k: int) -> bool:
    """
    Verify an Independent Set solution in polynomial time - O(k²).
    
    This is the "certifier" that proves Independent Set is in NP.
    Given a proposed subset, we check: (1) size >= k, (2) no edges between nodes.
    """
    if len(subset) < k:
        return False
    
    # Check that no two nodes in subset are connected
    subset_list = list(subset)
    for i, u in enumerate(subset_list):
        for v in subset_list[i + 1:]:
            if v in graph.get(u, set()):
                return False  # Found an edge - not independent!
    return True


# Demo: Verify SAT solution
clauses = [[1, -2], [-1, 3], [2, 3]]
good_assignment = {1: True, 2: True, 3: True}
bad_assignment = {1: False, 2: False, 3: False}

print("SAT Verification:")
print(f"  Clauses: {clauses}")
print(f"  Assignment {good_assignment}: {verify_sat(clauses, good_assignment)}")
print(f"  Assignment {bad_assignment}: {verify_sat(clauses, bad_assignment)}")

# Demo: Verify Independent Set
# Graph: 1--2--3--4 (a path)
graph = {1: {2}, 2: {1, 3}, 3: {2, 4}, 4: {3}}
good_subset = {1, 3}  # No edge between 1 and 3
bad_subset = {1, 2}   # Edge between 1 and 2

print("\nIndependent Set Verification:")
print(f"  Graph edges: 1-2, 2-3, 3-4")
print(f"  Subset {good_subset} with k=2: {verify_independent_set(graph, good_subset, 2)}")
print(f"  Subset {bad_subset} with k=2: {verify_independent_set(graph, bad_subset, 2)}")

### Reduction Example: Independent Set ↔ Vertex Cover

Independent Set and Vertex Cover are **equivalent** problems. Given a graph $G = (V, E)$:
- $S$ is an **independent set** if no two vertices in $S$ are adjacent
- $S$ is a **vertex cover** if every edge has at least one endpoint in $S$

**Key insight:** $S$ is an independent set of size $k$ if and only if $V - S$ is a vertex cover of size $n - k$.

In [ ]:
def verify_vertex_cover(graph: dict[int, set[int]], cover: set[int], k: int) -> bool:
    """
    Verify a Vertex Cover solution in polynomial time - O(|E|).
    Check that every edge has at least one endpoint in the cover.
    """
    if len(cover) > k:
        return False
    
    # Check every edge is covered
    for u, neighbors in graph.items():
        for v in neighbors:
            if u < v:  # Count each edge once
                if u not in cover and v not in cover:
                    return False  # Found uncovered edge!
    return True


def reduce_independent_set_to_vertex_cover(graph: dict[int, set[int]], k: int) -> tuple[dict, int]:
    """
    Reduction: Independent Set ≤_P Vertex Cover
    
    Transform: "Does G have independent set of size k?"
           → "Does G have vertex cover of size n-k?"
    
    This runs in O(1) time - we just transform the parameter!
    """
    n = len(graph)
    return graph, n - k


def convert_solution(vertices: set[int], independent_set: set[int]) -> set[int]:
    """Convert Independent Set solution to Vertex Cover solution."""
    return vertices - independent_set


# Demo the reduction
# Graph: triangle with extra node (1-2-3-1, plus 4 connected to 3)
graph = {1: {2, 3}, 2: {1, 3}, 3: {1, 2, 4}, 4: {3}}
vertices = {1, 2, 3, 4}
k_independent = 2

print("Reduction: Independent Set → Vertex Cover")
print(f"Graph: {graph}")
print(f"Question: Is there an independent set of size {k_independent}?")

# Transform the problem
_, k_cover = reduce_independent_set_to_vertex_cover(graph, k_independent)
print(f"Transformed: Is there a vertex cover of size {k_cover}?")

# The independent set {1, 4} works (no edges between them)
ind_set = {1, 4}
print(f"\nIndependent set found: {ind_set}")
print(f"  Valid? {verify_independent_set(graph, ind_set, k_independent)}")

# Convert to vertex cover
v_cover = convert_solution(vertices, ind_set)
print(f"Corresponding vertex cover: {v_cover}")
print(f"  Valid? {verify_vertex_cover(graph, v_cover, k_cover)}")

## Practical Applications <a class="anchor" id="applications"></a>

Understanding NP-completeness has significant real-world implications:

### Why Does This Matter?

**1. Cryptography** relies on computational hardness:
- RSA encryption depends on the difficulty of factoring large numbers
- If P = NP, most public-key cryptography would break instantly
- Hash functions, digital signatures, and secure communication all assume certain problems are hard

**2. Optimisation** in industry:
- **Scheduling:** Airline crew scheduling, job-shop scheduling → NP-complete
- **Logistics:** Vehicle routing, bin packing → NP-complete
- **Circuit design:** Component placement, wire routing → NP-complete
- Companies spend billions on approximate solutions to these problems

**3. AI and Machine Learning:**
- Many constraint satisfaction problems are NP-complete
- Training certain neural network architectures is NP-hard
- SAT solvers are used in formal verification of ML systems

### What To Do When Your Problem is NP-Complete

If you discover your problem is NP-complete, don't give up! You have options:

1. **Approximation algorithms:** Find solutions within a guaranteed factor of optimal
2. **Heuristics:** Use greedy algorithms, local search, genetic algorithms
3. **Special cases:** Many NP-complete problems have tractable special cases
4. **SAT solvers:** Modern solvers (MiniSat, Z3) can handle millions of variables for practical instances
5. **Parameterised complexity:** If some parameter $k$ is small, algorithms like $O(2^k \cdot n)$ may be practical

<details>
<summary><b>Comprehension Check:</b> If we could verify solutions in O(1) time instead of polynomial time, would this change the P vs NP question?</summary>

**Answer:** No! The P vs NP question would remain unchanged. The definition of NP requires polynomial-time verification, but changing verification to O(1) would just make NP smaller (fewer problems would qualify). The fundamental question of whether efficient solving implies efficient verification would still be open.

The key insight: NP is about the *existence* of polynomial-time verifiers, not about making verification as fast as possible.
</details>

## Exercises <a class="anchor" id="exercises"></a>

### Exercise 1: CNF Formulation

Convert the following constraints into a CNF formula and determine if it's satisfiable:

A group is planning a trip. The constraints are:
- At least one of Alice, Bob, or Carol must go: $(A \lor B \lor C)$
- If Alice goes, Bob must go: $(\neg A \lor B)$
- Bob and Carol cannot both go: $(\neg B \lor \neg C)$
- Carol must go: $(C)$

**Tasks:**
1. Write this as a single CNF formula
2. Is there a satisfying assignment? If so, who goes on the trip?

In [ ]:
# Exercise 1: Your solution here
# Variables: 1=Alice, 2=Bob, 3=Carol

exercise1_clauses = [
    # TODO: Fill in the clauses
]
num_vars_ex1 = 3

# Uncomment to test:
# result = brute_force_sat(exercise1_clauses, num_vars_ex1)
# print(f"Result: {result}")

<details>
<summary>Click for Solution</summary>

**CNF Formula:** $(A \lor B \lor C) \land (\neg A \lor B) \land (\neg B \lor \neg C) \land (C)$

**Analysis:**
- From clause 4: $C = True$ (Carol must go)
- From clause 3 with $C = True$: $B = False$ (Bob cannot go)
- From clause 2 with $B = False$: $A = False$ (Alice cannot go, since if she did, Bob would need to go)
- Check clause 1: $(False \lor False \lor True) = True$ ✓

**Answer:** Only Carol goes on the trip. Assignment: $A = False, B = False, C = True$

</details>

### Exercise 2: Problem Classification

Classify each of the following problems as **P**, **NP** (but not known to be in P), or **NP-complete**. Justify your answer.

1. **Sorting** a list of $n$ numbers
2. **3-SAT**: Given a CNF formula where each clause has exactly 3 literals, is it satisfiable?
3. **2-SAT**: Given a CNF formula where each clause has exactly 2 literals, is it satisfiable?
4. **Graph Coloring**: Can a graph be colored with $k$ colors such that no adjacent vertices share a color?
5. **Finding the shortest path** between two vertices in a weighted graph

<details>
<summary>Click for Solution</summary>

1. **Sorting** → **P**: Merge sort, quicksort run in $O(n \log n)$

2. **3-SAT** → **NP-complete**: This is THE canonical NP-complete problem (Cook-Levin theorem)

3. **2-SAT** → **P**: Can be solved in linear time using implication graphs and strongly connected components

4. **Graph Coloring** (for $k \geq 3$) → **NP-complete**: 3-coloring can be reduced from 3-SAT

5. **Shortest Path** → **P**: Dijkstra's algorithm runs in $O((V+E) \log V)$

</details>

### Exercise 3: Independent Set Verification

Given the following graph, find **all** maximum independent sets (independent sets of the largest possible size).

```
Graph: 
    1 --- 2
    |     |
    3 --- 4 --- 5
```

Edges: (1,2), (1,3), (2,4), (3,4), (4,5)

In [ ]:
# Exercise 3: Use verify_independent_set to check your answers
ex3_graph = {1: {2, 3}, 2: {1, 4}, 3: {1, 4}, 4: {2, 3, 5}, 5: {4}}

# Test candidate independent sets here:
# print(verify_independent_set(ex3_graph, {your_set}, k))

<details>
<summary>Click for Solution</summary>

**Maximum independent set size is 2.**

The maximum independent sets are:
- **{1, 5}**: No edge between 1 and 5
- **{2, 3}**: No edge between 2 and 3  
- **{2, 5}**: No edge between 2 and 5
- **{3, 5}**: No edge between 3 and 5

Why can't we get size 3? Node 4 is connected to nodes 2, 3, and 5. So any independent set containing 4 can only have 4 and 1. But 1 is connected to both 2 and 3. Any set of size 3 would need to include two nodes from {2, 3, 4, 5}, but we'd need to avoid all their edges.

</details>